In [14]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import calculations

In [15]:
headers = ["timestamp", "duration", "stability", "activity", "activity_confidence", "calibration_status", "w", "x", "y", "z"]
data = pd.read_csv('prototype_data/data.csv', names=headers)

data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
data['weekday'] = data['timestamp'].dt.day_name()
data['date'] = data['timestamp'].dt.date
data['hour'] = data['timestamp'].dt.hour
data['hour_minute'] = data['timestamp'].dt.hour + data['timestamp'].dt.minute / 60

data = data[data['timestamp'] >= pd.Timestamp('2024-01-01')]

data[['roll', 'pitch', 'yaw']] = data.apply(lambda row: calculations.quaternion_to_euler(row['w'], row['x'], row['y'], row['z']), axis=1, result_type='expand')

In [16]:
data

,timestamp,duration,stability,activity,activity_confidence,calibration_status,w,x,y,z,weekday,date,hour,hour_minute,roll,pitch,yaw
0,2024-06-24 08:05:20,157,In motion,In-Vehicle,92,2,0.366577,0.559509,0.626770,0.399597,Monday,2024-06-24,8,8.083333,114.323468,-0.708386,83.966903
1,2024-06-24 08:11:01,154,In motion,Still,74,2,0.362854,0.587463,0.627075,0.360474,Monday,2024-06-24,8,8.183333,118.489462,-1.807692,87.339682
2,2024-06-24 08:12:31,154,In motion,Still,92,3,-0.045166,-0.084228,0.411926,0.906189,Monday,2024-06-24,8,8.200000,49.398442,-6.629249,-8.758405
3,2024-06-24 08:16:12,186,In motion,Still,44,3,0.571533,0.243469,0.237366,0.746765,Monday,2024-06-24,8,8.266667,39.462557,5.296509,76.757793
4,2024-06-24 08:16:20,148,Stable,Still,74,3,0.341736,0.024719,0.283386,0.895691,Monday,2024-06-24,8,8.266667,32.040664,-8.592746,39.295561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4339,2024-07-09 13:57:00,153,In motion,Still,74,2,0.270203,0.161560,0.552124,0.772034,Tuesday,2024-07-09,13,13.950000,70.213720,-2.803573,36.607831
4340,2024-07-09 13:57:03,134,Stable,Still,81,2,0.354187,0.194458,0.510437,0.759094,Tuesday,2024-07-09,13,13.950000,66.159611,-3.804621,47.547922
4341,2024-07-09 13:57:06,162,Stable,Still,82,2,0.396912,0.217041,0.470947,0.757324,Tuesday,2024-07-09,13,13.950000,62.440942,-2.585454,53.750651
4342,2024-07-09 13:57:09,153,In motion,Still,86,2,0.070435,0.029968,0.611694,0.787354,Tuesday,2024-07-09,13,13.950000,75.521248,-2.233914,8.493411


In [17]:
filter_calibration_status = False
if filter_calibration_status:
    data = data[data['calibration_status'] >= 2]

In [18]:
all_activities = ['Unknown', 'In-Vehicle', 'On-Bicycle', 'On-Foot', 'Still', 'Tilting', 'Walking', 'Running', 'On Stairs']
activity_counts = data['activity'].value_counts().reindex(all_activities, fill_value=0)

activity_df = pd.DataFrame({'Activity': activity_counts.index, 'Number of Observations': activity_counts.values})

fig = px.bar(activity_df, x='Activity', y='Number of Observations', title='Activity Distribution', 
             labels={'Activity': 'Activity', 'Number of Observations': 'Number of Observations'}, 
             color='Activity')

fig.show()

In [19]:
all_stability_statuses = ['Unknown', 'On Table', 'Stationary', 'Stable', 'In motion']
stability_counts = data['stability'].value_counts().reindex(all_stability_statuses, fill_value=0)

stability_df = pd.DataFrame({'Stability': stability_counts.index, 'Number of Observations': stability_counts.values})

fig = px.bar(stability_df, x='Stability', y='Number of Observations', title='Stability Distribution', 
             labels={'Stability': 'Stability', 'Number of Observations': 'Number of Observations'}, 
             color='Stability')

fig.show()

In [20]:
all_calibration_statuses = [0, 1, 2, 3]
calibration_counts = data['calibration_status'].value_counts().reindex(all_calibration_statuses, fill_value=0)

calibration_df = pd.DataFrame({'Calibration Status': calibration_counts.index.astype(str), 'Number of Observations': calibration_counts.values})

fig = px.bar(calibration_df, x='Calibration Status', y='Number of Observations', title='Calibration Status Distribution', 
             labels={'Calibration Status': 'Calibration Status', 'Number of Observations': 'Number of Observations'}, 
             color='Calibration Status')

fig.show()

In [21]:
min_duration = data['duration'].min()
max_duration = data['duration'].max()
avg_duration = data['duration'].mean()

duration_table = pd.DataFrame({
    'Metric': ['Minimum Duration', 'Maximum Duration', 'Average Duration'],
    'Duration (ms)': [min_duration, max_duration, avg_duration]
})

duration_table

,Metric,Duration (ms)
0,Minimum Duration,79.000000
1,Maximum Duration,638.000000
2,Average Duration,166.996103


In [22]:
start_date = data['timestamp'].min().date()
end_date = data['timestamp'].max().date()
total_days_tracked = (end_date - start_date).days + 1

total_observations = len(data)

daily_counts = data['timestamp'].dt.date.value_counts().sort_index()
average_daily_observations = daily_counts.mean()

all_dates = pd.date_range(start=start_date, end=end_date).date
days_without_observations = len(set(all_dates) - set(daily_counts.index))

summary_table = pd.DataFrame({
    'Metric': ['Duration (days)', 'Number of Observations', 'Average Daily Observations', 'Days Without Observations'],
    'Value': [total_days_tracked, total_observations, average_daily_observations, days_without_observations]
})

summary_table

,Metric,Value
0,Duration (days),16.000000
1,Number of Observations,4106.000000
2,Average Daily Observations,456.222222
3,Days Without Observations,7.000000


In [23]:
fig = px.scatter(data, x='date', y='hour_minute', color='activity', title='Occurrences Per Day',
                 labels={'date': 'Date', 'hour_minute': 'Hour of the Day'}, height=800, size_max=10)

fig.update_xaxes(tickformat="%Y-%m-%d")
fig.update_yaxes(tickvals=list(range(0, 25)), autorange="reversed")
fig.update_layout(grid=dict(rows=1, columns=1, pattern="independent"))

fig.show()

In [24]:
# Classify observation based on Euler angles
def classify_observation(row):
    if 40 <= row['pitch'] <= 50:
        return 'Action 1'
    elif 5 <= row['pitch'] <= 15:
        return 'Action 2'
    else:
        return 'Other'

data.loc[:, 'observation'] = data.apply(classify_observation, axis=1)

fig = px.scatter(data, x='date', y='hour_minute', color='observation', title='Occurrences Per Day by Observation (Euler Angle Classification)',
                 labels={'date': 'Date', 'hour_minute': 'Hour of the Day'}, height=800, size_max=10)

fig.update_xaxes(tickformat="%Y-%m-%d")
fig.update_yaxes(tickvals=list(range(0, 25)), autorange="reversed")
fig.update_layout(grid=dict(rows=1, columns=1, pattern="independent"))

fig.show()

In [25]:
all_weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_counts = data['weekday'].value_counts().reindex(all_weekdays, fill_value=0)

average_weekday_counts = weekday_counts / len(data['timestamp'].dt.date.unique())

average_weekday_df = pd.DataFrame({'Weekday': average_weekday_counts.index, 'Average Number of Observations': average_weekday_counts.values})

fig = px.bar(average_weekday_df, x='Weekday', y='Average Number of Observations', title='Average Number of Observations per Weekday',
             labels={'Weekday': 'Weekday', 'Average Number of Observations': 'Average Number of Observations'}, 
             color='Weekday')

fig.show()

In [26]:
all_hours = range(24)
hourly_counts = data['hour'].value_counts().reindex(all_hours, fill_value=0)

total_observations = hourly_counts.sum()
hourly_percentage = (hourly_counts / total_observations) * 100

hourly_df = pd.DataFrame({'Hour': hourly_counts.index, 'Number of Observations': hourly_counts.values, 'Percentage': hourly_percentage.values})

fig = px.bar(hourly_df, x='Hour', y='Number of Observations', title='Number of Observations per Hour of the Day',
             labels={'Hour': 'Hour of the Day', 'Number of Observations': 'Number of Observations'}, 
             color='Hour')

for i in range(len(hourly_df)):
    fig.add_annotation(x=hourly_df['Hour'][i], y=hourly_df['Number of Observations'][i], 
                       text=f'{hourly_df["Percentage"][i]:.1f}%', showarrow=False, 
                       yshift=10, font=dict(size=10, color='black'))

fig.update_layout(xaxis_tickmode='linear')
fig.show()